In [1]:
%%writefile portfolio_module.py
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import norm, skew, kurtosis
from scipy.stats.mstats import gmean
import datetime

def compute_relative_power_sharpe(returns_series):
    if returns_series.empty:
        return np.nan
    target_value = np.log(4)
    def objective(P_X):
        return (np.mean(np.log(1 + (returns_series / P_X)**2)) - target_value)**2
    initial_guess = 1.0
    result = minimize(objective, initial_guess, bounds=[(1e-9, None)])
    if result.success and not np.isnan(result.x[0]):
        P_X = float(result.x[0])
        mean_return = returns_series.mean()
        return (mean_return / P_X) if P_X > 0 else np.nan
    else:
        return np.nan

def objective_sharpe(weights, daily_ret):
    portfolio_return = np.dot(weights, daily_ret.mean()) * 252
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(daily_ret.cov() * 252, weights)))
    return -(portfolio_return / (portfolio_volatility + 1e-12))

def objective_cvar(weights, daily_ret):
    port_ret = np.dot(daily_ret, weights)
    mean_annual = port_ret.mean() * 252
    std_annual  = port_ret.std()  * np.sqrt(252)
    conf_level = 0.05
    cvar_metric = mean_annual - std_annual * norm.ppf(conf_level)
    return -cvar_metric

def objective_sortino(weights, daily_ret):
    port_daily_annualized = np.dot(daily_ret, weights) * 252
    downside = port_daily_annualized[port_daily_annualized < 0]
    if len(downside) == 0:
        return -1e9
    downside_std = downside.std()
    sortino_ratio = port_daily_annualized.mean() / (downside_std + 1e-12)
    return -sortino_ratio

def objective_variance(weights, daily_ret):
    cov = daily_ret.cov() * 252
    return np.dot(weights.T, np.dot(cov, weights))

def objective_relative_power_sharpe(weights, daily_ret):
    port_returns = pd.Series(np.dot(daily_ret, weights), index=daily_ret.index)
    rps = compute_relative_power_sharpe(port_returns)
    return -rps

def dynamic_investment(full_rets, study_start_date, study_end_date, invest_start_date, invest_end_date,
                       lookback_years, freq, obj_func, init_capital, symbols):
    # Set up initial guess, bounds, and constraints based on number of assets
    n = len(symbols)
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((-1, 1) for _ in range(n))
    init_guess = np.array([1.0 / n] * n)

    # Create a date index for the investment period
    dates_idx = pd.date_range(start=invest_start_date, end=invest_end_date, freq=freq)
    if dates_idx[-1] < pd.to_datetime(invest_end_date):
        dates_idx = dates_idx.append(pd.DatetimeIndex([pd.to_datetime(invest_end_date)]))
    all_dates = []
    all_values = []
    capital = init_capital
    current_weights = None

    for i in range(1, len(dates_idx)):
        rebal_date = dates_idx[i]
        prev_date = dates_idx[i - 1]
        # Ensure the lookback period does not start before the study period begins:
        lookback_start = max(pd.to_datetime(study_start_date), rebal_date - pd.DateOffset(years=lookback_years))
        window_data = full_rets.loc[lookback_start: rebal_date - pd.Timedelta(days=1)]
        if len(window_data) == 0:
            continue
        result = minimize(lambda w: obj_func(w, window_data),
                          init_guess, method='SLSQP',
                          bounds=bounds, constraints=cons)
        new_weights = result.x if result.success else init_guess

        if current_weights is None:
            current_weights = new_weights

        daily_range = full_rets.loc[prev_date: rebal_date - pd.Timedelta(days=1)]
        if len(daily_range) > 0:
            port_ret_series = daily_range.dot(current_weights)
            growth = (1 + port_ret_series).cumprod()
            sub_port_values = capital * growth
            for dt, val in sub_port_values.items():
                all_dates.append(dt)
                all_values.append(val)
            capital = sub_port_values.iloc[-1]
        current_weights = new_weights

    out_series = pd.Series(data=all_values, index=all_dates).sort_index()
    return out_series

def run_simulation(study_start_date, study_end_date, invest_start_date, invest_end_date,
                   symbols, objective_choice='Sharpe', initial_investment=10000,
                   lookback_years=2, rebalance_freq='Q'):
    # Download full data from study_start_date to invest_end_date.
    full_data = yf.download(symbols, start=study_start_date, end=invest_end_date)['Close']
    full_returns = full_data.pct_change().dropna()

    # Set up available objective functions.
    objectives = {
        'Sharpe': objective_sharpe,
        'CVaR': objective_cvar,
        'Sortino': objective_sortino,
        'Variance': objective_variance,
        'PowerSharpe': objective_relative_power_sharpe
    }
    if objective_choice not in objectives:
        selected_objective = objective_sharpe
    else:
        selected_objective = objectives[objective_choice]

    # Run the dynamic investment simulation.
    result_series = dynamic_investment(full_returns, study_start_date, study_end_date,
                                       invest_start_date, invest_end_date, lookback_years,
                                       rebalance_freq, selected_objective, initial_investment, symbols)
    final_value = result_series.iloc[-1] if len(result_series) > 0 else None

    # Create a plot.
    fig, ax = plt.subplots(figsize=(10, 6))
    if len(result_series) > 0:
        ax.plot(result_series.index, result_series, label=f"Dynamic {objective_choice}")
    else:
        ax.text(0.5, 0.5, "No rebalancing data produced.", horizontalalignment='center',
                verticalalignment='center', transform=ax.transAxes)
    ax.set_title("Dynamic Portfolio Value Over Time")
    ax.set_xlabel("Date")
    ax.set_ylabel("Portfolio Value ($)")
    ax.legend()
    ax.grid(True)

    return final_value, fig

Writing portfolio_module.py


In [2]:
%%writefile ranking_module.py
import numpy as np
import pandas as pd
import yfinance as yf
import seaborn as sns
from scipy.stats import norm, t, skewnorm, gennorm, entropy
import matplotlib.pyplot as plt

# -----------------------------
# Risk Measure Functions
# -----------------------------
def compute_standard_sharpe(returns, trading_days_per_year=252):
    returns = returns.squeeze()
    if returns.empty or returns.std() == 0:
        return np.nan
    return (returns.mean() / returns.std()) * np.sqrt(trading_days_per_year)

def compute_relative_power_sharpe(returns):
    returns = returns.squeeze()
    if returns.empty:
        return np.nan
    target_value = np.log(4)
    annual_factor = np.sqrt(252)
    def objective(P_X):
        return (np.mean(np.log(1 + (returns / P_X) ** 2)) - target_value) ** 2
    from scipy.optimize import minimize
    result = minimize(objective, 1.0, bounds=[(0, None)])
    if result.success and not np.isnan(result.x[0]):
        P_X = float(result.x[0])
        return (np.mean(np.log(1 + returns)) / P_X) * annual_factor if P_X > 0 else np.nan
    else:
        return np.nan

def compute_snr_sharpe(returns, trading_days_per_year=252):
    returns = returns.squeeze()
    if returns.empty:
        return np.nan
    log_returns = np.log(1 + returns)
    mean_lr = log_returns.mean()
    std_error = log_returns.std() / np.sqrt(len(log_returns))
    if std_error == 0:
        return np.nan
    return (mean_lr / std_error) * np.sqrt(trading_days_per_year)

def compute_median_sharpe(returns, trading_days_per_year=252):
    returns = returns.squeeze()
    if returns.empty:
        return np.nan
    med = np.median(returns)
    mad = np.median(np.abs(returns - med))
    if mad == 0:
        return np.nan
    return (med / mad) * np.sqrt(trading_days_per_year)

def compute_sortino_ratio(returns, trading_days_per_year=252):
    returns = returns.squeeze()
    if returns.empty:
        return np.nan
    mean_ret = returns.mean()
    neg_returns = returns[returns < 0]
    if neg_returns.std() == 0:
        return np.nan
    return (mean_ret / neg_returns.std()) * np.sqrt(trading_days_per_year)

def compute_max_drawdown(returns):
    returns = returns.squeeze()
    if returns.empty:
        return np.nan
    cum_returns = (1 + returns).cumprod()
    peak = cum_returns.cummax()
    dd = (cum_returns - peak) / peak
    return dd.min()

def compute_information_ratio(returns, benchmark_returns, trading_days_per_year=252):
    returns = returns.squeeze()
    benchmark_returns = benchmark_returns.squeeze()
    aligned = pd.concat([returns, benchmark_returns], axis=1).dropna()
    if aligned.empty:
        return np.nan
    ex_ret = aligned.iloc[:, 0] - aligned.iloc[:, 1]
    if ex_ret.std() == 0:
        return np.nan
    return (ex_ret.mean() / ex_ret.std()) * np.sqrt(trading_days_per_year)

def compute_mvar(returns, confidence_level=0.95):
    returns = returns.squeeze()
    if returns.empty:
        return np.nan
    mean_ret = returns.mean()
    std_dev = returns.std()
    z = norm.ppf(confidence_level)
    return mean_ret + z * std_dev

def compute_evar(returns, alpha=0.95):
    returns = returns.squeeze()
    if returns.empty:
        return np.nan
    z_vals = np.linspace(0.01, 10, 100)
    mgf = lambda t: np.mean(np.exp(t * returns))
    evar_vals = [(1 / t) * (np.log(mgf(t)) - np.log(alpha)) for t in z_vals if mgf(t) > 0]
    return min(evar_vals) if evar_vals else np.nan

def compute_rlvar(returns, alpha=0.95, kappa=0.5):
    returns = returns.squeeze()
    if returns.empty:
        return np.nan
    mgf_kappa = lambda t: np.mean((1 + t * returns / kappa) ** kappa)
    z_vals = np.linspace(0.01, 10, 100)
    rlvar_vals = [(1 / t) * (np.log(mgf_kappa(t)) - np.log(alpha)) for t in z_vals if mgf_kappa(t) > 0]
    return min(rlvar_vals) if rlvar_vals else np.nan

def compute_ulcer_index(returns):
    returns = returns.squeeze()
    if returns.empty:
        return np.nan
    cum_returns = (1 + returns).cumprod()
    peak = cum_returns.cummax()
    dd = (cum_returns - peak) / peak
    return np.sqrt(np.mean(dd ** 2))

# -----------------------------
# Main Function: ranking_plot
# -----------------------------
def ranking_plot(symbols, static_start, static_end, benchmark_symbol):
    """
    For a given list of stock symbols and a benchmark,
    fetches historical data between static_start and static_end,
    computes risk metrics for each stock, computes unified integer risk rankings,
    creates a heatmap figure (without the color bar), and returns that figure.

    Parameters:
      - symbols (list): List of stock tickers.
      - static_start (str): Start date ('YYYY-MM-DD').
      - static_end (str): End date ('YYYY-MM-DD').
      - benchmark_symbol (str): Benchmark ticker.

    Returns:
      - fig (matplotlib.figure.Figure): Heatmap figure of unified risk rankings.
    """
    import numpy as np
    import pandas as pd
    import yfinance as yf
    import matplotlib.pyplot as plt
    import seaborn as sns

    # Fetch historical data and compute returns for each stock.
    returns_data = {}
    for symbol in symbols:
        data = yf.download(symbol, start=static_start, end=static_end, progress=False)
        if data.empty:
            print(f"No data for {symbol}")
            continue
        price_series = data['Adj Close'] if 'Adj Close' in data.columns else data['Close']
        returns = price_series.pct_change().dropna()
        if returns.empty:
            print(f"No returns data for {symbol}")
            continue
        returns_data[symbol] = returns

    # Fetch benchmark data.
    benchmark_data = yf.download(benchmark_symbol, start=static_start, end=static_end, progress=False)
    if benchmark_data.empty:
        raise ValueError("No benchmark data available.")
    benchmark_series = benchmark_data['Adj Close'] if 'Adj Close' in benchmark_data.columns else benchmark_data['Close']
    benchmark_returns = benchmark_series.pct_change().dropna()

    # Define risk measure functions.
    risk_measures = {
        'Standard Sharpe': lambda r: compute_standard_sharpe(r),
        'Relative Power Sharpe': lambda r: compute_relative_power_sharpe(r),
        'SNR Sharpe': lambda r: compute_snr_sharpe(r),
        'Median Sharpe': lambda r: compute_median_sharpe(r),
        'Sortino Ratio': lambda r: compute_sortino_ratio(r),
        'Maximum Drawdown': lambda r: compute_max_drawdown(r),
        'Information Ratio': lambda r: compute_information_ratio(r, benchmark_returns),
        'MVaR': lambda r: compute_mvar(r),
        'EVaR': lambda r: compute_evar(r),
        'RLVaR': lambda r: compute_rlvar(r),
        'Ulcer Index': lambda r: compute_ulcer_index(r)
    }

    # Compute metrics for each risk measure.
    metrics_dict = {name: {} for name in risk_measures.keys()}
    for symbol, returns in returns_data.items():
        for name, func in risk_measures.items():
            try:
                metrics_dict[name][symbol] = func(returns)
            except Exception:
                metrics_dict[name][symbol] = np.nan

    metrics_df = pd.DataFrame(metrics_dict)

    # Invert metrics for which higher is better so that lower values imply lower risk.
    metrics_higher_is_better = ['Standard Sharpe', 'Relative Power Sharpe', 'SNR Sharpe', 'Median Sharpe', 'Sortino Ratio', 'Information Ratio']
    risk_scores = metrics_df.copy()
    for metric in risk_scores.columns:
        if metric in metrics_higher_is_better:
            risk_scores[metric] = -risk_scores[metric]

    unified_risk_ranking = risk_scores.rank(ascending=True, method='average')
    # Convert to integer ranks.
    unified_risk_ranking = unified_risk_ranking.round().astype(int)

    # Create heatmap figure without the color bar.
    fig, ax = plt.subplots(figsize=(12, 6))
    sns.heatmap(unified_risk_ranking, annot=True, fmt="d", cmap="coolwarm", linewidths=0.5, ax=ax, cbar=False)
    ax.set_title('Static Stock Rankings (1 = Least Risk)', fontsize=16)
    ax.set_xlabel('Risk Metrics', fontsize=14)
    ax.set_ylabel('Stocks', fontsize=14)
    plt.tight_layout()

    return fig


Writing ranking_module.py


In [3]:
%%writefile rolling_basis_module.py

import numpy as np
import pandas as pd
import yfinance as yf
import numpy as np
import pandas as pd
import yfinance as yf

# import each of your measure-functions:
from ranking_module import (
    compute_standard_sharpe,
    compute_sortino_ratio,
    compute_median_sharpe,
    compute_snr_sharpe,
    compute_relative_power_sharpe,
    compute_max_drawdown,
    compute_information_ratio,
    compute_mvar,
    compute_evar,
    compute_rlvar,
    compute_ulcer_index,
)

# Map human-friendly names → functions
RISK_FUNCS = {
    'Standard Sharpe': compute_standard_sharpe,
    'Sortino Ratio': compute_sortino_ratio,
    'Median Sharpe': compute_median_sharpe,
    'SNR Sharpe': compute_snr_sharpe,
    'Power Sharpe': compute_relative_power_sharpe,
    'Max Drawdown': compute_max_drawdown,
    'Information Ratio': compute_information_ratio,
    'MVaR': compute_mvar,
    'EVaR': compute_evar,
    'RLVaR': compute_rlvar,
    'Ulcer Index': compute_ulcer_index,
}

def compute_rolling_risk(ticker, start_date, end_date, window=126,
                         measures=None, benchmark=None):
    """
    If measures is None, compute all in RISK_FUNCS.
    If benchmark is needed by some measures (like info ratio), pass it here.
    """
    df = yf.download(ticker, start=start_date, end=end_date, progress=False)
    price = df.get('Adj Close', df['Close'])
    daily_ret = price.pct_change().dropna()

    dates = daily_ret.index[window-1:]
    # decide which metrics to compute
    measures = measures or list(RISK_FUNCS.keys())
    risk_df = pd.DataFrame(index=dates, columns=measures, dtype=float)

    # if info ratio needs a benchmark series pre-computed:
    if 'Information Ratio' in measures and benchmark is not None:
        benchmark_price = benchmark.get('Adj Close', benchmark['Close'])
        benchmark_ret = benchmark_price.pct_change().dropna()
    else:
        benchmark_ret = None

    for dt in dates:
        window_ret = daily_ret.loc[:dt].tail(window)
        for name in measures:
            func = RISK_FUNCS[name]
            if name == 'Information Ratio':
                risk_df.at[dt, name] = func(window_ret, benchmark_ret)
            else:
                risk_df.at[dt, name] = func(window_ret)

    # compute rankings if you want to return those too
    rank_df = risk_df.rank(ascending=True, method='average')
    return risk_df, rank_df


Writing rolling_basis_module.py


In [4]:
%%writefile app.py
import streamlit as st
from datetime import datetime
import matplotlib.pyplot as plt
import rolling_basis_module    as rbm
import portfolio_module as pm
import ranking_module as rm
import yfinance as yf
import pandas as pd
import numpy as np


# Define a common list of stock options.
st.set_page_config(layout="wide")

st.title("Trading in Extremistan")

stock_options = ["AAPL","TSLA","JPM","WMT","JNJ","PEP","DIS","LMT","T","GOOGL"]

tabs = st.tabs([
    "Dynamic Portfolio Simulation",
    "Static Stock Ranking",
    "Distribution Fitting",
    "Rolling-Basis Risk"
])

# ---------------- Dynamic Simulation Tab ----------------
with tabs[0]:
    st.markdown("""
    ### Dynamic Portfolio Simulation
    Use the controls below to set the training (study) and testing periods, and select up to 8 stock tickers.
    """)
    study_start = st.date_input("Study Start Date (Training)", datetime(2009, 1, 1), key="dyn_start")
    study_end = st.date_input("Study End Date (Training) & Investment Start Date (Testing)", datetime(2019, 12, 1), key="dyn_end")
    invest_end = st.date_input("Investment End Date (Testing)", datetime(2020, 9, 1), key="dyn_invest_end")

    selected_tickers = st.multiselect("Select up to 8 Stock Tickers", options=stock_options,
                                      default=["AAPL", "TSLA", "JPM", "WMT", "JNJ", "PEP", "DIS", "LMT"], key="dyn_tickers")
    if len(selected_tickers) > 8:
        st.error("Please select at most 8 tickers.")

    objective_choice = st.selectbox("Select Objective Function",
                                    options=["Sharpe", "CVaR", "Sortino", "Variance", "PowerSharpe"], key="dyn_obj")

    if st.button("Run Dynamic Simulation", key="dyn_run"):
        if len(selected_tickers) > 8:
            st.error("Too many tickers selected. Please choose 8 or fewer.")
        else:
            study_start_date = study_start.strftime("%Y-%m-%d")
            study_end_date = study_end.strftime("%Y-%m-%d")
            invest_start_date = study_end_date  # As required.
            invest_end_date = invest_end.strftime("%Y-%m-%d")

            st.info("Running simulation. This may take a few moments...")
            final_value, fig = pm.run_simulation(
                study_start_date, study_end_date,
                invest_start_date, invest_end_date,
                selected_tickers,
                objective_choice=objective_choice,
                initial_investment=10000,
                lookback_years=2,
                rebalance_freq='Q'
            )
            if final_value is not None:
                st.success(f"Final Portfolio Value: ${final_value:,.2f}")
            else:
                st.error("Simulation did not produce any results. Check parameters and data availability.")
            st.pyplot(fig)

# ---------------- Static Ranking Tab ----------------
with tabs[1]:
    st.markdown("""
    ### Static Stock Ranking
    This demo computes unified risk rankings (displayed as a heatmap) across a full static period.
    """)
    static_start = st.date_input("Static Period Start Date", datetime(2018, 1, 1), key="stat_start")
    static_end = st.date_input("Static Period End Date", datetime(2021, 9, 30), key="stat_end")
    default_stocks = ["NEM", "GOLD", "APA"]
    ranking_symbols = st.multiselect("Select Stocks for Ranking",
                                     options=["NEM", "GOLD", "APA", "TSLA", "AAPL", "JPM"],
                                     default=default_stocks, key="stat_symbols")
    # Benchmark is hardcoded to "^GSPC" and not shown.
    benchmark_symbol = "^GSPC"

    if st.button("Show Ranking Heatmap", key="stat_run"):
        s_start = static_start.strftime("%Y-%m-%d")
        s_end = static_end.strftime("%Y-%m-%d")
        st.info("Fetching data and computing rankings. Please wait...")
        try:
            fig = rm.ranking_plot(ranking_symbols, s_start, s_end, benchmark_symbol)
            st.pyplot(fig)
        except Exception as e:
            st.error(f"Error computing ranking: {e}")

# ---------------- Distribution Fitting Tab ----------------
with tabs[2]:
    st.markdown("### Distribution Fitting (Gaussian vs Student-t)")
    fit_stock = st.selectbox("Select Stock for Fitting Analysis", options=stock_options, key="fit_stock")
    fit_start = st.date_input("Fitting Analysis Start Date", datetime(2020, 1, 1), key="fit_start")
    fit_end = st.date_input("Fitting Analysis End Date", datetime(2020, 12, 31), key="fit_end")

    if st.button("Run Distribution Fitting", key="fit_run"):
         fit_start_date = fit_start.strftime("%Y-%m-%d")
         fit_end_date = fit_end.strftime("%Y-%m-%d")
         data_fit = yf.download(fit_stock, start=fit_start_date, end=fit_end_date, progress=False)
         if data_fit.empty:
             st.error("No data fetched for the selected stock.")
         else:
             price_series_fit = data_fit['Adj Close'] if 'Adj Close' in data_fit.columns else data_fit['Close']
             returns_fit = price_series_fit.pct_change().dropna()
             if returns_fit.empty:
                 st.error("No returns data computed.")
             else:
                 from scipy.stats import norm, t
                 norm_params = norm.fit(returns_fit)
                 t_params = t.fit(returns_fit)

                 fig_fit, ax_fit = plt.subplots(figsize=(10, 6))
                 ax_fit.hist(returns_fit, bins=50, density=True, alpha=0.6, color="gray", label="Empirical")
                 x_fit = np.linspace(returns_fit.min(), returns_fit.max(), 1000)
                 ax_fit.plot(x_fit, norm.pdf(x_fit, *norm_params), label="Gaussian Fit", color="blue", lw=2)
                 ax_fit.plot(x_fit, t.pdf(x_fit, *t_params), label="Student-t Fit", color="red", lw=2)
                 ax_fit.legend()
                 ax_fit.set_title(f"Distribution Fitting for {fit_stock}")
                 st.pyplot(fig_fit)

# ----------- Rolling-Basis Risk & Ranking Evolution -----------
with tabs[3]:
    st.header("Rolling-Basis Risk & Ranking Evolution")

    ticker_rb = st.selectbox("Select Ticker", stock_options, index=0, key="rbm_ticker")
    start_rb  = st.date_input("Start Date", datetime(2018, 1, 1), key="rbm_start")
    end_rb    = st.date_input("End Date",   datetime(2021, 9, 30), key="rbm_end")
    window_rb = st.slider("Rolling Window (days)", 30, 252, 126, key="rbm_window")

    all_measures = list(rbm.RISK_FUNCS.keys())
    chosen = st.multiselect("Which risk measures?", all_measures, default=["Standard Sharpe"])

    if st.button("Compute Rolling Risk", key="rbm_run"):
        # if you need a benchmark series for info ratio:
        benchmark = yf.download("^GSPC",
                               start=start_rb.strftime("%Y-%m-%d"),
                               end=end_rb.strftime("%Y-%m-%d"),
                               progress=False)

        risk_df, rank_df = rbm.compute_rolling_risk(
            ticker_rb,
            start_rb.strftime("%Y-%m-%d"),
            end_rb.strftime("%Y-%m-%d"),
            window=window_rb,
            measures=chosen,
            benchmark=benchmark
        )

        st.subheader("Raw Rolling Risk Measures")
        st.line_chart(risk_df[chosen], use_container_width=True)

        st.subheader("Rolling-Rankings")
        st.line_chart(rank_df[chosen], use_container_width=True)


Writing app.py


In [5]:
print()
"""
with tabs[x]:
    st.header("Trading-Strategy Simulations")
    ticker = st.selectbox("Select Ticker", stock_options, index=0, key="tsm_ticker")
    start_date = st.date_input("Start Date", datetime(2018, 1, 1), key="tsm_start")
    end_date   = st.date_input("End Date",   datetime(2021, 9, 30), key="tsm_end")

    if st.button("Run Simulations", key="tsm_run"):
        df_results, cum_series = tsm.run_trading_simulations(
            ticker,
            start_date.strftime("%Y-%m-%d"),
            end_date.strftime("%Y-%m-%d")
        )

        st.subheader("Summary Results")
        st.dataframe(df_results)

        st.subheader("Cumulative Returns Over Time")
        for name, series in cum_series.items():
            st.line_chart(series, height=250, use_container_width=True)
"""

'\nwith tabs[x]:\n    st.header("Trading-Strategy Simulations")\n    ticker = st.selectbox("Select Ticker", stock_options, index=0, key="tsm_ticker")\n    start_date = st.date_input("Start Date", datetime(2018, 1, 1), key="tsm_start")\n    end_date   = st.date_input("End Date",   datetime(2021, 9, 30), key="tsm_end")\n\n    if st.button("Run Simulations", key="tsm_run"):\n        df_results, cum_series = tsm.run_trading_simulations(\n            ticker,\n            start_date.strftime("%Y-%m-%d"),\n            end_date.strftime("%Y-%m-%d")\n        )\n\n        st.subheader("Summary Results")\n        st.dataframe(df_results)\n\n        st.subheader("Cumulative Returns Over Time")\n        for name, series in cum_series.items():\n            st.line_chart(series, height=250, use_container_width=True)\n'

In [6]:
!pip install streamlit pyngrok > /dev/null

In [7]:
import os
import time
from pyngrok import ngrok
ngrok.set_auth_token("YOUR AUTHTOKEN")  # authtoken

os.system("pkill streamlit")
os.system("pkill -f ngrok")

time.sleep(3)

In [8]:
# Open an ngrok tunnel on port 8501.
public_url = ngrok.connect(8501, "http")

# Launch the Streamlit app in the background.
os.system("streamlit run app.py &")

time.sleep(5)
print("Your Streamlit app is available at:", public_url)

Your Streamlit app is available at: NgrokTunnel: "https://4e63-35-243-173-244.ngrok-free.app" -> "http://localhost:8501"
